In [3]:
import pandas as pd
import os

# Combine scoring table to box scores from SR for single year

In [2]:
### Combine scoring table to box scores from SR for single year

# Load your single-row-per-game DataFrame and scoring details DataFrame
nfl_data_single_row = pd.read_csv('./data-raw-sr/box-scores/v3_cleaned_all_nfl_box_scores_2022.csv')
nfl_scoring_data = pd.read_csv('./data-raw-sr/scoring-tables/all_nfl_scoring_tables_2022.csv')

# Find the maximum number of scoring events in a single game
max_scores_in_game = nfl_scoring_data.groupby('Game_ID').size().max()

# Initialize a list to store the new game data with re-used scoring columns
optimized_new_game_columns_reuse = []

# Loop through each unique 'Game_ID'
for game_id in nfl_data_single_row['Game_ID'].unique():
    # Initialize a dictionary to store this game's data
    game_scores = {f'Score_{i+1}_{key}': float('nan') for i in range(max_scores_in_game) for key in ['Q', 'Time', 'Team', 'Details']}
    
    # Get the scoring data for this game
    game_scoring_data = nfl_scoring_data[nfl_scoring_data['Game_ID'] == game_id]
    
    # Fill in the actual scores
    for i, (_, row) in enumerate(game_scoring_data.iterrows()):
        game_scores[f'Score_{i+1}_Q'] = row['Quarter']
        game_scores[f'Score_{i+1}_Time'] = row['Time']
        game_scores[f'Score_{i+1}_Team'] = row['Team']
        game_scores[f'Score_{i+1}_Details'] = row['Detail']
        
    # Add this game's data to the list
    optimized_new_game_columns_reuse.append(game_scores)

# Convert the list of dictionaries to a DataFrame
optimized_new_game_columns_df_reuse = pd.DataFrame(optimized_new_game_columns_reuse)

# Merge this DataFrame with the original one
nfl_data_combined_single_row_optimized_reuse = pd.concat([nfl_data_single_row, optimized_new_game_columns_df_reuse], axis=1)

# Save the DataFrame to a CSV file
nfl_data_combined_single_row_optimized_reuse.to_csv('./data-final/box_scores_with_scorers.csv', index=False)


In [4]:
### Loop it

# Define the directory containing the CSV files
input_directory = './data-raw-sr/box-scores/'
scoring_directory = './data-raw-sr/scoring-tables/'
output_directory = './data-final/'

# Loop through each file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.csv') and filename.startswith('v3_cleaned'):  # Filter by extension and prefix
        input_file_path = os.path.join(input_directory, filename)
        
        # Extract the year from the filename to find the corresponding scoring table
        year = filename.split('_')[-1].split('.csv')[0]
        scoring_file_path = os.path.join(scoring_directory, f'all_nfl_scoring_tables_{year}.csv')

        if os.path.exists(scoring_file_path):  # Make sure the corresponding scoring file exists
            # Load your single-row-per-game DataFrame and scoring details DataFrame
            nfl_data_single_row = pd.read_csv(input_file_path)
            nfl_scoring_data = pd.read_csv(scoring_file_path)

            # Find the maximum number of scoring events in a single game
            max_scores_in_game = nfl_scoring_data.groupby('Game_ID').size().max()

            # Initialize a list to store the new game data with re-used scoring columns
            optimized_new_game_columns_reuse = []

            # Loop through each unique 'Game_ID'
            for game_id in nfl_data_single_row['Game_ID'].unique():
                # Initialize a dictionary to store this game's data
                game_scores = {f'Score_{i+1}_{key}': float('nan') for i in range(max_scores_in_game) for key in ['Q', 'Time', 'Team', 'Details']}

                # Get the scoring data for this game
                game_scoring_data = nfl_scoring_data[nfl_scoring_data['Game_ID'] == game_id]

                # Fill in the actual scores
                for i, (_, row) in enumerate(game_scoring_data.iterrows()):
                    game_scores[f'Score_{i+1}_Q'] = row['Quarter']
                    game_scores[f'Score_{i+1}_Time'] = row['Time']
                    game_scores[f'Score_{i+1}_Team'] = row['Team']
                    game_scores[f'Score_{i+1}_Details'] = row['Detail']

                # Add this game's data to the list
                optimized_new_game_columns_reuse.append(game_scores)

            # Convert the list of dictionaries to a DataFrame
            optimized_new_game_columns_df_reuse = pd.DataFrame(optimized_new_game_columns_reuse)

            # Merge this DataFrame with the original one
            nfl_data_combined_single_row_optimized_reuse = pd.concat([nfl_data_single_row, optimized_new_game_columns_df_reuse], axis=1)

            # Save the DataFrame to a CSV file in the output directory
            output_file_path = os.path.join(output_directory, f'box_scores_with_scorers_{year}.csv')
            nfl_data_combined_single_row_optimized_reuse.to_csv(output_file_path, index=False)


# MERGE BOX SCORES w/ SCORE DETAILS TO NFL_GAMES_FINAL.CSV

In [9]:
### Make Standard Game_ID column in nfl_games_final.csv

# Load the nfl_games_final CSV file
nfl_games_df = pd.read_csv('./data-final/nfl_games_final.csv')

# Create the Game_ID column based on the given format
nfl_games_df['Game_ID'] = nfl_games_df['schedule_season'].astype(str) + '_' + \
                          nfl_games_df['schedule_week'].astype(str).str.zfill(2) + '_' + \
                          nfl_games_df['away_team_id'].astype(str) + '_' + \
                          nfl_games_df['home_team_id'].astype(str)

# Save the updated DataFrame to a new CSV file
nfl_games_df.to_csv('./data-final/nfl_games_final_with_game_id.csv', index=False)

print("Game_IDs created and saved to 'nfl_games_final_with_game_id.csv'.")


Game_IDs created and saved to 'nfl_games_final_with_game_id.csv'.


In [12]:
### Perform the merge (just 2022)

# Load the updated CSV files
nfl_games_df = pd.read_csv('./data-final/nfl_games_final_with_game_id.csv')
nfl_games_2022_df = pd.read_csv('./data-final/box_scores_with_scorers_2022.csv')

# Merge the datasets using the 'Game_ID' from nfl_games_df and 'Game_ID_Standard' from nfl_games_2022_df
merged_df = pd.merge(nfl_games_df, nfl_games_2022_df, left_on='Game_ID', right_on='Game_ID_Standard', how='left')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('./data-final/merged_nfl_games.csv', index=False)

print("Datasets merged and saved to 'merged_nfl_games.csv'.")


Datasets merged and saved to 'merged_nfl_games.csv'.


In [16]:
### Perform the merge (loop) TESTING

# Load the main dataset
nfl_games_df = pd.read_csv('./data-final/nfl_games_final_with_game_id.csv')

# Initialize merged_df to be the same as nfl_games_df
merged_df = nfl_games_df.copy()

# Directory containing all the other years' box score CSV files
box_scores_dir = './data-final/'

# Iterate through each CSV file in the directory
for filename in os.listdir(box_scores_dir):
    if filename.endswith('.csv') and filename.startswith('box_scores_with_scorers'):
        # Full path to the CSV file
        filepath = os.path.join(box_scores_dir, filename)

        # Load the box score CSV file for the current year
        current_year_df = pd.read_csv(filepath)

        # Merge with the main dataset using 'left' join
        merged_df = pd.merge(merged_df, current_year_df, left_on='Game_ID', right_on='Game_ID_Standard', how='left', suffixes=('', '_'+filename[:-4]))

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('./data-final/merged_nfl_games.csv', index=False)

print("Datasets merged and saved to 'merged_nfl_games.csv'.")


Datasets merged and saved to 'merged_nfl_games.csv'.


In [14]:
### Clean some columns up 

# Load the merged CSV file
merged_df = pd.read_csv('./data-final/merged_nfl_games.csv')

# Drop the specified unnecessary columns
columns_to_drop = ['season', 'week', 'Game_ID_Standard', 'Team1_ABBR', 'Team2_ABBR', 'Game_ID_y']
cleaned_merged_df = merged_df.drop(columns=columns_to_drop)

# Rename 'Game_ID_x' to 'Game_ID' for clarity
cleaned_merged_df = cleaned_merged_df.rename(columns={'Game_ID_x': 'Game_ID'})

# Save the cleaned DataFrame to a new CSV file
cleaned_merged_df.to_csv('./data-final/cleaned_merged_nfl_games.csv', index=False)

print("Unnecessary columns removed and cleaned DataFrame saved to 'cleaned_merged_nfl_games.csv'.")


/var/folders/hd/r4k9j4m15vd1bc_tnk5pn5_c0000gn/T/ipykernel_2483/1783053805.py:4: DtypeWarning: Columns (10,25,26,27,28,39,40,41,42,43,46,47,49,50,51,64,65,66,67,69,70,71,73,74,75,77,78,79,81,82,83,85,86,87,89,90,91,92,93,94,95,97,98,99,100,101,102,103,105,106,107,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,125,126,127,128,129,130) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('./data-final/merged_nfl_games.csv')


Unnecessary columns removed and cleaned DataFrame saved to 'cleaned_merged_nfl_games.csv'.
